In [1]:
%pip install plotly
%pip install nbformat>=4.2.0
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

Note: you may need to restart the kernel to use updated packages.
zsh:1: 4.2.0 not found
Note: you may need to restart the kernel to use updated packages.


### 1.4 razy idea followed by a stupid one

a) Given a matrix $X$ and a vector $y$ as in the two tables above, implement
python/numpy code that first solves the least squares problem
\begin{align*}
    w_\star = \arg\min_{w \in \mathcal{R}^3} \left\lVert X w - y \right\rVert^2
\end{align*}
and then computes
\begin{align*}
    \hat{y} = X w_\star
\end{align*}

In [3]:
import numpy.linalg as la

def compute_lstsq_y_hat(X, y):
    #compute w_star with the built-in least-sqares function from numpy
    w_star = la.lstsq(X, y)[0]
    #compute y_hat with the result from lstsq
    return X @ w_star

b) Run your code for rules $110$ and $126$ and print the respective vectors $y$ and $\hat{y}$. 

In [4]:
# chatgpt helped writing out rules

XT = np.array([
    [+1, +1, +1],
    [+1, +1, -1],
    [+1, -1, +1],
    [+1, -1, -1],
    [-1, +1, +1],
    [-1, +1, -1],
    [-1, -1, +1],
    [-1, -1, -1]
])

y_rule_110 = np.array([+1, -1, -1, -1, +1, -1, -1, +1])

y_rule_126 = np.array([+1, -1, -1, -1, -1, -1, -1, +1])

# Solve the least squares problem for rule 110
y_hat_110 = compute_lstsq_y_hat(XT, y_rule_110)
y_hat_126 = compute_lstsq_y_hat(XT, y_rule_126)


# Create dataframes for better comparison of y and y_hat
results_110 = pd.DataFrame({
    'Original y (Rule 110)': y_rule_110,
    'Predicted y_hat (Rule 110)': y_hat_110
})

results_126 = pd.DataFrame({
    'Original y (Rule 126)': y_rule_126,
    'Predicted y_hat (Rule 126)': y_hat_126
})

# Print the results in table format for better alignment and comparison
print("Results for Rule 110:")
print(results_110)
print()

print("Results for Rule 126:")
print(results_126)

Results for Rule 110:
   Original y (Rule 110)  Predicted y_hat (Rule 110)
0                      1                        0.25
1                     -1                       -0.25
2                     -1                       -0.25
3                     -1                       -0.75
4                      1                        0.75
5                     -1                        0.25
6                     -1                        0.25
7                      1                       -0.25

Results for Rule 126:
   Original y (Rule 126)  Predicted y_hat (Rule 126)
0                      1                2.285890e-16
1                     -1               -1.195986e-16
2                     -1                2.285890e-16
3                     -1               -1.195986e-16
4                     -1                1.195986e-16
5                     -1               -2.285890e-16
6                     -1                1.195986e-16
7                      1               -2.285890e-16


In [5]:
# Function to visualize points in 3D space for better understanding of non-linearity using Plotly
def plot_3d_representation(XT, y, title):
    # Define colors based on y values
    colors = ['blue' if val == 1 else 'red' for val in y]

    # Create a scatter plot in 3D
    trace = go.Scatter3d(
        x=XT[:, 0],
        y=XT[:, 1],
        z=XT[:, 2],
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            opacity=0.8,
            symbol='circle'
        ),
        text=[f'y={val}' for val in y]
    )

    layout = go.Layout(
        title=title,
        scene=dict(
            xaxis_title='X1',
            yaxis_title='X2',
            zaxis_title='X3'
        )
    )

    fig = go.Figure(data=[trace], layout=layout)
    pio.show(fig)

# Plotting for Rule 110 and Rule 126
plot_3d_representation(XT, y_rule_110, "Rule 110: 3D Representation")
plot_3d_representation(XT, y_rule_126, "Rule 126: 3D Representation")

Analyze your results. Do they suggest that rules $110$ and $126$ are linear functions $y = f(x)$ or not? Discuss this in your own words.

They cannot be linear in the 3 arguments, since the values of y hat are significantly off.


> What we do here is attempting to fit a linear model to the rules. We aim to find a predictor function (linear in the 3 input values of the neighborhood) $g(x_0, x_2, x_3)$ that is a best-fit approximation of the output of the rule. We use each possible state configuration with their the target output as sample points.
>
> We use the binary transform $f_{+-}$ for this to ensure a centering, such that the zero function is in the center.
>
>The least squares solution is just adding up the weights from the output with respect to the input. That is, multiply each column with the target column and add up (and then normalize).
>
> We could imagine this as a cube where each corner votes (either positive or negative) if a point is at its location and then interpolating their locations (see plot).
>
> Looking at it the other way around: If we multiply $\hat{y}$ with the current observation of the state of the cellular automaton, we get the likelihoods with regard to its previous state.
>
> For the second table we get zero as a result since the rule is symmetric with regard to flipping all signs.